### Regression with huber loss function

Функция потерь Хьюбера
$$
L_\delta(a,y)=
\begin{cases}
 \frac{1}{2}(y - a)^2,                   & |y - a| \le \delta, \\
 \delta\, |y - a| - \frac{1}{2}\delta^2 & \textrm{иначе.}
\end{cases}
$$
производная по вектору 

$$
\frac{\partial L}{\partial\omega}=\left\{\begin{array}{l}X^T(y\;-\;\omega X)\;,\;\;\;\;\;\left|y\;-\;\omega X\right|\leqslant\delta\\X^Tsign\lbrack y-\omega X\rbrack\end{array}\right.
$$

In [1]:
import pandas as pd
import numpy.linalg as la
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
class HuberReg(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='stochastic',
                 tolerance=1e-4, max_iter=10000, w0=None, alpha=1e-3,batch_size = 10):
        """
        gd_type: 'full' or 'stochastic'
        tolerance: for stopping gradient descent
        max_iter: maximum number of steps in gradient descent
        w0: np.array of shape (d) - init weights
        alpha: momentum coefficient
        """
        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0  
        self.alpha = alpha
        self.w = None
        self.batch_size = batch_size
        self.loss_history = None 
    
    def calc_loss(self, X, y):
        if la.norm(y - np.dot(X,self.w)) <= self.delta:
            return 0.5 * la.norm(y -np.dot(X,self.w))
        else:
            return self.delta*la.norm((y - np.dot(X,self.w) - 0.5*self.delta) , ord = 1)
    
    def calc_gradient(self, X, y):
        
        if self.gd_type != 'full' and self.gd_type != 'stochastic':
            raise Exception('Unknown gd_type')
            
        if self.gd_type == 'full':
            if (la.norm(y - np.dot(X,self.w))) <= self.delta:
                grad = np.dot(X.T, (np.dot(X,self.w) - y)) / y.shape[0]
            else:
                grad =  self.delta *np.dot(X.T, np.sign(np.dot(X,self.w) - y))/ y.shape[0]
            
        if self.gd_type == 'stochastic':
            sample = np.random.randint(low = 0, high = X.shape[0],size = self.batch_size)                
            if la.norm(y - np.dot(X,self.w)) <= self.delta:
                grad = np.dot(X.iloc[sample].T, (np.dot(X.iloc[sample],self.w) - y.iloc[sample]))/self.batch_size
            else:
                grad =  self.delta *np.dot(X.iloc[sample].T,
                                               np.sign(np.dot(X.iloc[sample],self.w) -y.iloc[sample]))/self.batch_size
        return grad


    def fit(self, X, y):
        self.loss_history = []
        step_size_0 = 0.06
        self.w = np.zeros(X.shape[1])
        w_mem = self.w.copy()
        h = np.zeros(X.shape[1])
        for i in range(self.max_iter):
            step_size = step_size_0 / ((i+1)**0.51)
            self.w -=h
            self.loss_history.append(self.calc_loss(X,y))
            h = self.alpha * h + step_size *self.calc_gradient(X,y)
            if np.abs(la.norm(w_mem) - la.norm(self.w)) < self.tolerance and i != 0:
                print('Stoped by w-norm')
                break

        return self
    
    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        return np.dot(X,self.w)
        pass
    
    def score(self, X, y):
        if self.w is None:
            raise Exception('Not trained yet')
        return (1 - ((y - np.dot(X,self.w))**2).sum()/((y - np.mean(y))**2).sum())

### Example

In [3]:
data=pd.read_csv('data.csv')
data.head()

,f0,f1,f2,f3,f4,f5,f6
0,16.99,1.01,0.976270,-3.697815,0.623295,0.524760,7199.992000
1,10.34,1.66,4.303787,7.715073,0.886961,0.473862,2466.136700
2,21.01,3.50,2.055268,-6.464284,0.618826,1.657394,2969.369100
3,23.68,3.31,0.897664,1.335254,0.133461,1.234554,1040.665300
4,24.59,3.61,-1.526904,-0.196414,0.980580,3.086397,37.469975


In [4]:
# normolaize the features 
for i in range(0,7):
    data['f%s' % i ] = (data['f%s' % i ] - data['f%s' % i].mean()) / data['f%s' % i].var()
X_train, X_test, y_train, y_test = train_test_split(data, data['f1'],test_size = 0.3)

In [5]:
%%time
hubreg = HuberReg(gd_type='full')
hubreg.fit(X_train,y_train)
print("R2-score: %s" %hubreg.score(X_test,y_test))
print ("W: %s" %hubreg.w)

R2-score: 0.9998967797035738
W: [ 9.25342178e-02  9.88099841e-01 -2.44870755e-03 -1.41178595e-02
 -2.78762425e-04  8.38814277e-04 -3.95176991e-06]
CPU times: user 11.3 s, sys: 0 ns, total: 11.3 s
Wall time: 11.3 s


In [6]:
%%time
hubreg = HuberReg(gd_type='stochastic')
hubreg.fit(X_train,y_train)
print("R2-score: %s" %hubreg.score(X_test,y_test))
print ("W: %s" %hubreg.w)

R2-score: 0.9998920181042534
W: [ 9.46874861e-02  9.87938072e-01 -8.24309624e-03 -1.27391517e-02
 -2.27232360e-04  7.70048161e-04 -9.98237841e-07]
CPU times: user 19 s, sys: 1.52 ms, total: 19 s
Wall time: 19 s


In [7]:
%%time
params = {'alpha':np.arange(0.8,2,0.1)}
grid = GridSearchCV(hubreg,param_grid=params, cv = 5)
grid.fit(X_train, y_train)

/home/hacket/anaconda3/lib/python3.7/site-packages/numpy/linalg/linalg.py:2386: RuntimeWarning: overflow encountered in reduce
  return add.reduce(abs(x), axis=axis, keepdims=keepdims)
/home/hacket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: overflow encountered in subtract
/home/hacket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: overflow encountered in multiply


CPU times: user 21min 43s, sys: 344 ms, total: 21min 44s
Wall time: 21min 44s


In [8]:
print('R2-score: %s' %grid.score(X_test,y_test))

R2-score: 1.0
